In [1]:
import folium
from folium import Tooltip
import geopandas as gpd
import pandas as pd

In [2]:
# Read the shapefiles
continents = gpd.read_file("/kaggle/input/game-of-thrones-atlas/GoTRelease/continents.shp")
westeros = gpd.read_file("/kaggle/input/game-of-thrones-atlas/GoTRelease/political.shp")
islands = gpd.read_file("/kaggle/input/game-of-thrones-atlas/GoTRelease/islands.shp")
locations = gpd.read_file("/kaggle/input/game-of-thrones-atlas/GoTRelease/locations.shp")
rivers = gpd.read_file("/kaggle/input/game-of-thrones-atlas/GoTRelease/rivers.shp")
lakes = gpd.read_file("/kaggle/input/game-of-thrones-atlas/GoTRelease/lakes.shp")
roads = gpd.read_file("/kaggle/input/game-of-thrones-atlas/GoTRelease/roads.shp")
wall = gpd.read_file("/kaggle/input/game-of-thrones-atlas/GoTRelease/wall.shp")

In [3]:
# Custom styles for each layer
def land_style(feature):
    return {
        'fillColor': 'wheat', 
        'color': 'black', 
        'weight': 1, 
        'fillOpacity': 0.8
    }

def water_style(feature):
    return {
        'fillColor': '#a6cee3', 
        'color': '#a6cee3', 
        'weight': 1, 
        'fillOpacity': 1
    }

def river_style(feature):
    return {
        'color': '#a6cee3', 
        'weight': 1,
        'opacity': 1
    }

def road_style(feature):
    return {
        'color': 'gray',
        'weight': 1.5, 
        'dashArray': '5, 5'
    }

def wall_style(feature):
    return {
        'color': 'dimgray', 
        'weight': 2.5
    }

In [4]:
# Calculate center of the map
all_geometries = gpd.GeoSeries(pd.concat([
    continents.geometry,
    islands.geometry,
    westeros.geometry,
    lakes.geometry,
    rivers.geometry,
    roads.geometry,
    wall.geometry,
    locations.geometry
]))
centroid = all_geometries.unary_union.centroid
center = [centroid.y, centroid.x]

In [5]:
# Create a map
m = folium.Map(location=center, zoom_start=3.5, tiles=None, control_scale=True)

# Inject CSS to change the background color of the map
m.get_root().html.add_child(folium.Element("""
<style>
    .leaflet-container {
        background-color: #add8e6;  /* Light blue background color */
    }
</style>
<link href="https://fonts.googleapis.com/css2?family=MedievalSharp&display=swap" rel="stylesheet">
"""))

# Add layers
folium.GeoJson(continents, style_function=land_style).add_to(m)
folium.GeoJson(islands, style_function=land_style).add_to(m)
folium.GeoJson(westeros, style_function=land_style).add_to(m)
folium.GeoJson(lakes, style_function=water_style).add_to(m)
folium.GeoJson(rivers, style_function=river_style).add_to(m)
folium.GeoJson(roads, style_function=road_style).add_to(m)
folium.GeoJson(wall, style_function=wall_style).add_to(m)

# Add markers
for _, row in locations.iterrows():
    coords = row.geometry.centroid.coords[0]
    styled_tooltip = Tooltip(
        f"<div style='font-family: MedievalSharp; font-size: 12px; color: black;'>{row['name']}</div>",
        sticky=True,
        direction="top"
    )
    circle_marker = folium.CircleMarker(
        location=[coords[1], coords[0]],
        radius=2,
        color='maroon',
        fill=True,
        fill_opacity=1,
        tooltip=styled_tooltip
    ).add_to(m)

# Add political region names as text
for _, row in westeros.iterrows():
    # Get the centroid of the political region
    centroid = row.geometry.centroid
    lat, lon = centroid.y, centroid.x
    region_name = row['name']
    folium.Marker(
        location=[lat, lon],
        icon=folium.DivIcon(
            html=f'<div style="font-family: MedievalSharp, cursive; font-size: 18px; color: black; font-weight: bold; opacity: 0.8">{region_name}</div>'
        ),
    ).add_to(m)

# Add continents text
for _, row in continents.iterrows():
    centroid = row.geometry.centroid
    lat, lon = centroid.y, centroid.x
    continent_name = row['name']
    folium.Marker(
        location=[lat, lon],
        icon=folium.DivIcon(
            html=f'<div style="font-family: MedievalSharp, cursive; font-size: 30px; color: black; font-weight: bold; opacity: 0.4">{continent_name}</div>'
        ),
    ).add_to(m)

m